Suponga que tiene un robot que le entrega galletas al grupo de ingenieros de diseño de robots. Programe por PG el recorrido del robot, teniendo en cuenta que cada vez que un ingeniero recibe una galleta gana puntos. Los ingenieros están distribuidos en una sala cuadrada. Defina, conjunto de terminales, conjunto de funciones y función de aptitud.



In [1]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 5.1 MB/s eta 0:00:00


In [52]:
import random
import os
import time
from deap import base, creator, tools, gp, algorithms
import numpy as np

# ======= CONFIGURACIÓN =======
GRID_SIZE = 5
engineers = {(1, 1), (3, 2), (0, 4), (2, 2)}
State = tuple  # (pos, visited, path)

# ======= MOVIMIENTOS =======
def move_up(pos): return (max(0, pos[0] - 1), pos[1])
def move_down(pos): return (min(GRID_SIZE - 1, pos[0] + 1), pos[1])
def move_left(pos): return (pos[0], max(0, pos[1] - 1))
def move_right(pos): return (pos[0], min(GRID_SIZE - 1, pos[1] + 1))

def _move(pos, visited, path, move_func):
    new_pos = move_func(pos)
    visited = visited.copy()
    visited.add(new_pos)
    path = path.copy()
    path.append(new_pos)
    return (new_pos, visited, path)

def up(pos, visited, path): return _move(pos, visited, path, move_up)
def down(pos, visited, path): return _move(pos, visited, path, move_down)
def left(pos, visited, path): return _move(pos, visited, path, move_left)
def right(pos, visited, path): return _move(pos, visited, path, move_right)
def stay(pos, visited, path): return (pos, visited.copy(), path + [pos])

def seq(a, b):
    def combined(pos, visited, path):
        pos, visited, path = a(pos, visited, path)
        return b(pos, visited, path)
    return combined

# ======= PRIMITIVE SETUP =======
pset = gp.PrimitiveSetTyped("MAIN", [], State)
pset.addTerminal(up, State, name="UP")
pset.addTerminal(down, State, name="DOWN")
pset.addTerminal(left, State, name="LEFT")
pset.addTerminal(right, State, name="RIGHT")
pset.addTerminal(stay, State, name="STAY")  # puedes comentar esta línea si quieres eliminar STAY
pset.addPrimitive(seq, [State, State], State)

# ======= DEAP SETUP =======
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("expr", gp.genFull, pset=pset, min_=2, max_=5)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

# ======= FUNCIÓN DE APTITUD =======
def eval_robot(ind):
    func = toolbox.compile(expr=ind)
    pos = (2, 1)
    visited = set([pos])
    path = [pos]
    try:
        pos, visited, path = func(pos, visited, path)
    except:
        return 0.,
    engineers_visited = visited & engineers
    repeats = sum(1 for i in range(1, len(path)) if path[i] == path[i-1])
    penalty = repeats * 0.05  # penalización por quedarse quieto
    score = len(engineers_visited) + 0.1 * len(visited) - penalty
    return max(score, 0.),

toolbox.register("evaluate", eval_robot)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr, pset=pset)

# ======= SIMULACIÓN COMPLETA =======
def simular_recorrido_completo(ind):
    func = toolbox.compile(expr=ind)
    pos = (2, 1)
    visited = set([pos])
    path = [pos]
    try:
        pos, visited, path = func(pos, visited, path)
    except:
        pass
    return path, visited

# ======= ANIMACIÓN EN CONSOLA =======
def mostrar_matriz(path, engineers, grid_size=5, delay=0.2):
    for step, pos in enumerate(path):
        os.system('cls' if os.name == 'nt' else 'clear')
        print(f"🧭 Paso {step}/{len(path) - 1}")
        for x in range(grid_size):
            row = ""
            for y in range(grid_size):
                cell = (x, y)
                if cell == pos:
                    row += " R "
                elif cell in engineers:
                    row += " E "
                elif cell in path[:step]:
                    row += " · "
                else:
                    row += " . "
            print(row)
        time.sleep(delay)

# ======= EJECUCIÓN PRINCIPAL =======
def run_gp():
    pop = toolbox.population(n=100)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values[0])
    stats.register("avg", np.mean)
    stats.register("max", np.max)

    pop, _ = algorithms.eaSimple(
        pop, toolbox, cxpb=0.4, mutpb=0.6, ngen=40,
        stats=stats, halloffame=hof, verbose=True
    )

    best = hof[0]
    print("\n🏆 Mejor individuo:")
    print(best)
    print(f"Puntaje: {best.fitness.values[0]:.2f}")

    path, visited = simular_recorrido_completo(best)

    print("\n🧭 Detalle del recorrido del mejor robot:")
    for i, p in enumerate(path):
        print(f"Paso {i:2d}: {p}")
    print(f"\nIngenieros visitados: {visited & engineers}")
    print(f"Total de ingenieros: {len(visited & engineers)}")

    mostrar_matriz(path, engineers)

# ======= EJECUTAR =======
if __name__ == "__main__":
    run_gp()


gen	nevals	avg  	max 
0  	100   	1.788	4.55
1  	82    	2.554	5.25
2  	79    	3.253	5.25
3  	74    	3.7285	5.4 
4  	82    	4.1935	5.4 
5  	74    	4.5755	5.6 
6  	82    	4.567 	5.6 
7  	75    	4.666 	5.6 
8  	86    	4.613 	5.6 
9  	72    	4.7185	5.6 
10 	73    	4.8695	5.6 
11 	81    	4.723 	5.7 
12 	77    	4.813 	5.7 
13 	81    	4.8575	5.75
14 	79    	5.0295	5.75
15 	73    	5.1095	5.75
16 	71    	4.996 	5.65
17 	78    	4.9635	5.65
18 	74    	5.0275	5.65
19 	79    	5.012 	5.65
20 	74    	5.007 	5.7 
21 	75    	5.0735	5.75
22 	78    	5.1085	5.75
23 	77    	5.092 	5.75
24 	79    	5.1145	5.9 
25 	71    	5.2475	5.9 
26 	71    	5.2095	5.9 
27 	75    	5.2865	5.85
28 	73    	5.2685	5.85
29 	73    	5.2465	5.85
30 	81    	5.1295	5.85
31 	83    	5.1785	5.95
32 	82    	5.1875	5.95
33 	76    	5.234 	5.9 
34 	73    	5.305 	5.9 
35 	80    	5.2425	6   
36 	73    	5.251 	6   
37 	79    	5.349 	6   
38 	81    	5.3125	5.95
39 	81    	5.345 	5.95
40 	81    	5.323 	6   

🏆 Mejor individuo:
seq(seq(UP, seq(se